In [1]:
from __future__ import division
from pyomo.environ import *
import numpy as np
import pandas as pd
import time
from functools import reduce
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
from pprint import pprint
import itertools

In [149]:
product_data = pd.read_csv('data/AnnualOptimizerInputData.csv')
last_quarter_product_base = pd.read_csv('data/LastQuarterData.csv')
coefficient_matrix = pd.read_csv('data/coefficient_matrix.csv')


products = product_data.shape[0]
idx = range(products)
weeks = 52
tot = products*weeks

init_data = pd.read_csv('data/Weekly_optimiser_2_results.csv')

# coefficient_matrix = coefficient_matrix.head(6)
# init_data = init_data.head(6)    ### first weeek only

init_data['edlp_discount'] = init_data['EDLP_Price']/init_data['BasePrice']
init_data['BasePrice'] = init_data['BasePrice'].astype(float)

In [152]:
base_price = []
for i in range(products):
    base_price.append(init_data.iloc[i::6, 9].values.tolist())
    
edlp = []
for i in range(products):
    edlp.append(init_data.iloc[i::6, 12].values.tolist())

tpr = []
for i in range(products):
    tpr.append(init_data.iloc[i::6, 8].values.tolist())

flag = []
for i in range(products):
    flag.append(init_data.iloc[i::6, 5].values.tolist())

In [153]:
coef = []
coefficient_matrix_dr = coefficient_matrix.drop(columns=['Modelled_Product','wk'])#'Product1_Retailer_EDLP_Product6_Retailer_EDLP'])

for i in range(products):
    coef.append(coefficient_matrix_dr.iloc[i::6, :].values.tolist())

In [154]:
target_trade_spend = product_data.Product_Total_Spend.tolist()
total_target_trade_spend_final = sum(target_trade_spend)
target_edlp_spend = product_data.Product_EDLP_Spend.tolist()
target_tpr_spend = product_data.Product_TPR_Spend.tolist()
target_tpr_events = product_data.TPR_Events.tolist()

target_trade_spend_lower_bd = [x * 0.9 for x in target_trade_spend]
target_trade_spend_upper_bd = [x * 1.1 for x in target_trade_spend]

target_edlp_spend_lower_bd = [x * 0.8 for x in target_edlp_spend]
target_edlp_spend_upper_bd = [x * 1.2 for x in target_edlp_spend]

target_tpr_spend_lower_bd = [x * 0.8 for x in target_tpr_spend]
target_tpr_spend_upper_bd = [x * 1.2 for x in target_tpr_spend]

target_tpr_events_lb = product_data.TPR_Events_Min.tolist()
target_tpr_events_ub = product_data.TPR_Events_Max.tolist()

In [197]:
def calculate_interaction(prd_no, inter_prd, edlp, flag,i):

    p1 = base_price[prd_no][i] * (1-flag[prd_no][i]) + base_price[prd_no][i]* edlp[prd_no][i] * (flag[prd_no][i])
    p2 = base_price[inter_prd][i] * (1-flag[inter_prd][i]) + base_price[inter_prd][i]* edlp[inter_prd][i] * (flag[inter_prd][i])
    
    return np.log(p1)*np.log(p2)

def cal_sales_prior(edlp,tpr,flag,coef,j,factors):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        total = 0
        cc= 0
        for p in range(products):
            if p==j:
                z = 0
                edlp_sum+= (np.log( base_price[p][i] * edlp[p][i])*coef[i][cc+1])* z
                tpr_sum+= (np.log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p][i]*z)*(1- z)
            else:
                edlp_sum+= (np.log( base_price[p][i] * edlp[p][i])*coef[i][cc+1])* flag[p][i]
                tpr_sum+= (np.log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p][i]*coef[i][cc])*(1- flag[p][i])
                
            cc+=2
            
        try:
            interaction = calculate_interaction(j, factors[j], edlp,flag,i)
        except:
            interaction = 1
            
        unit_sales.append(np.exp(edlp_sum+ tpr_sum + coef[i][-2] + interaction*coef[i][-1]))                 
    return unit_sales
    
def cal_sales(edlp,tpr,flag,coef,j,factors):
    unit_sales = []
    for i in  range(weeks):
        edlp_sum = 0
        tpr_sum = 0
        cc= 0
        for p in range(products):
#             print('base_price= ',base_price[p][i], ' edlp= ',edlp[p][i] , ' edlp_coef= ', coef[i][cc+1], 
#                   ' flag=' , flag[p][i], ' tpr= ', tpr[p][i], ' tpr_coef= ', coef[i][cc])
            
            edlp_sum+= (np.log( base_price[p][i] * edlp[p][i])*coef[i][cc+1])* flag[p][i]
            tpr_sum+= (np.log(base_price[p][i])*coef[i][cc+1]+ 100*tpr[p][i]*coef[i][cc])*(1- flag[p][i])
            cc+=2
        try:
            interaction = calculate_interaction(j, factors[j], edlp,flag,i)
        except:
            interaction = 1
#         print('interaction=', np.exp(interaction*coef[i][-1]))
        unit_sales.append(np.exp(edlp_sum+ tpr_sum + coef[i][-2] + interaction*coef[i][-1]))            
#     print()
    return unit_sales

def calc_price(edlp,tpr,flag,j):
    price = []
    for i in range(weeks):
        price.append((base_price[j][i]*(1-tpr[j][i])*(1-flag[j][i]))  +  (base_price[j][i]*edlp[j][i]*flag[j][i]))
    return price

def calc_dollar_sales(edlp,tpr,flag,coef,interaction_factors, prior=False):
    dollar_sales=[]
    price = []
    unit_sales = []
    
    for j in range(products):
        if not prior:
            unit_sales_single= cal_sales(edlp,tpr,flag,coef[j],j,interaction_factors)
            price_single = calc_price(edlp,tpr,flag,j)
        else:
            unit_sales_single = cal_sales_prior(edlp,tpr,flag,coef[j],j,interaction_factors)
            price_single = base_price[j]

        dollar_sales_single = [unit_sales_single[i]*price_single[i] for i in range(weeks)]
        dollar_sales.append(dollar_sales_single)
        price.append(price_single)
        unit_sales.append(unit_sales_single)
    
    return dollar_sales, unit_sales, price

def calc_total_trade_spent(edlp,tpr,flag,coef, unit_sales, price):
    total_trade_spent = []
    for j in range(products):
        trade_spent = [base_price[j][i]-price[j][i] for i in range(weeks)]
        total_trade_spent_partial = [trade_spent[i]*unit_sales[j][i] for i in range(weeks)]
        total_trade_spent.append(total_trade_spent_partial)
        
    return total_trade_spent

def calc_edlp_trade_spent(edlp,tpr,flag,coef,total_trade_spent):
    edlp_trade_spent = []
#     flag = [flag_p1,flag_p2]
    for j in range(products):
        edlp_trade_spent_partial = [total_trade_spent[j][i]*flag[j][i] for i in range(weeks)]
        edlp_trade_spent.append(edlp_trade_spent_partial)
    return edlp_trade_spent


def calculate_lift(dollar_sales, prior_prom_sales):
    lift = []
    for i in range(products):
        for j in range(weeks):
            lift.append(dollar_sales[i][j] - prior_prom_sales[i][j])
    return lift

def calculate_total_tpr_events(flag):
    tpr_events = []
    for i in range(products):
        temp_sum = 0
        for j in range(weeks):
            temp_sum+=flag[i][j]
        tpr_events.append(weeks-temp_sum)
    return tpr_events

def calculate_tpr_trade_spend(total_trade_spent, edlp_trade_spent):
    tpr_spend =[]
    for i in range(products):
        tpr_spend.append([a-b for a,b in zip(total_trade_spent[i], edlp_trade_spent[i])])
    return tpr_spend

In [208]:
weeks = 52
interaction_factors = {}
interaction_factors[0] = 5
interaction_factors[5] = 0

# for k in range(products):
#     print(k,'-------------')
#     print('res=',cal_sales(edlp,tpr,flag,coef[k],k,interaction_factors))
    
initial,unit_sales_1, price_1 = calc_dollar_sales(edlp,tpr,flag,coef,interaction_factors,True)
final,unit_sales_2,price_2 = calc_dollar_sales(edlp,tpr,flag,coef,interaction_factors)
t = calc_total_trade_spent(edlp,tpr,flag,coef,unit_sales_2,price_2)

d_s = []
t_sp = []
for i in range(weeks):
    for j in range(products):
        d_s.append(final[j][i])
        t_sp.append(t[j][i])
# print(result)
pd.DataFrame(data=list(zip(d_s, t_sp)), columns =['Dollar_sales', 'Total_Spend'])

,Dollar_sales,Total_Spend
0,11835.069373,59.773078
1,36456.747382,362.153120
2,10011.191137,0.000000
3,35718.178065,0.000000
4,27233.127272,400.487166
...,...,...
307,37778.646088,375.284564
308,10185.440422,0.000000
309,35905.578331,0.000000
310,26223.861371,385.645020
